In [1]:
%pip install torch
%pip install torchvision



[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: C:\Users\jepey\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.1.2 -> 22.3
[notice] To update, run: C:\Users\jepey\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import random
import numpy as np
np.random.seed(309)
random.seed(309)
torch.manual_seed(309)

In [64]:
transform = transforms.Compose(
    [transforms.ToTensor(), #converts to pytorch tensor
     transforms.Resize((250,250)), #Changed to 250x250 size
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) #normalize the images mean 0.5 and std 0.5
transformFlip = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomHorizontalFlip(p=1), #adds horizontal flip to all images
     transforms.Resize((250,250)), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transformVerFlip = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomVerticalFlip(p=1), #adds vertical flip to all images
     transforms.Resize((250,250)), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


#Uses file with outliers removed
#Loads training file and transforms it
train = torchvision.datasets.ImageFolder(root="C:/Users/jepey/OneDrive/Documents/COMP309/ProData/traindata",transform=transform)
#Loads training file and transforms it with horizontal flipping
trainF = torchvision.datasets.ImageFolder(root="C:/Users/jepey/OneDrive/Documents/COMP309/ProData/traindata",transform=transformFlip)
#Loads training file and transforms it with vertical flipping
trainV = torchvision.datasets.ImageFolder(root="C:/Users/jepey/OneDrive/Documents/COMP309/ProData/traindata",transform=transformVerFlip)
valT, trainT = torch.utils.data.random_split(
    train, (int(len(train)*0.3),len(train)-int(len(train)*0.3)) #Gets unaugmented validation data of size 20% of whole set
)
useF, trashF = torch.utils.data.random_split(
    trainF, (int(len(trainF)*0.25),len(trainF)-int(len(trainF)*0.25)) #Gets 25% of the horizontal flipped images to use
)
useV, trashV = torch.utils.data.random_split(
    trainV, (int(len(trainV)*0.25),len(trainV)-int(len(trainV)*0.25)) #Gets 25% of the vertical flipped images to use
)

fullTrain = trainT + useF + useV


trainLoader = torch.utils.data.DataLoader(fullTrain, batch_size=64,
                                          shuffle=True)
valLoader = torch.utils.data.DataLoader(valT, batch_size=64,
                                          shuffle=True)
print(len(fullTrain))
print(len(trainLoader))
print(len(valLoader))
# classes for computing accuracy
classes = ('cherry','strawberry','tomato')

6643
21


Function for computing training accuracy

In [30]:
#Compares actual labels to labels predicted
def accuracy(predictions, labels):
    #For each row picks index with highest value as class eg 0,1,2
    classes = torch.argmax(predictions, dim=1)
    #If prediction is correct add to counter
    return torch.mean((classes == labels).float())

MLP Validation Accuracy Function

In [ ]:
def getValScore(valLoader):
  correct = 0
  total = 0
  # Since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in valLoader: #Iterates through validation set
          images, labels = data
          # calculate outputs by running images through the network
          outputs = mlp(images)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          #Adds to total counter
          total += labels.size(0) 
          #If correct add to counter
          correct += (predicted == labels).sum().item()
  return correct/total

MLP Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        #3 linear layers to make the classifications to 3 classes
        self.fc1 = nn.Linear(250*250*3, 120) #After flattening take each RGB value gor every pixel and reduce it to 120
        self.fc2 = nn.Linear(120, 84) #Reduce further to 84
        self.fc3 = nn.Linear(84, 3) #Reduce to number of classes being 3

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        #print(x.size()) for testing
         #Applies 3 linear layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x)) 
        x = self.fc3(x)
        #Returns flattening as batch to be used for classification
        return x

#Creates initial model
mlp = MLP()

Initialize loss criteria and optimizer

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.001, momentum=0.9)

Optimizes model and computes accuracies and loss at each epoch

In [ ]:
epochList = []
valList = []
accList = []
lossList = []
for epoch in range(30):  # loop over the dataset 30 times
    running_accuracy=0.0
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients  as they typically accumulate
        optimizer.zero_grad()

        # forward + backward + optimize
        #Get outputs from current model
        outputs = mlp(inputs)
        #Calculate Loss
        loss = criterion(outputs, labels)
        #Runs autograd to fill in gradients of optimizer using loss
        loss.backward()
        #Updates optimizer parameters
        optimizer.step()

        # print statistics
        
        _, predicted = torch.max(outputs.data, 1)
        running_accuracy += accuracy(outputs, labels) 
    running_loss += loss.item()
    vScore = getValScore(valLoader)
    print("Epoch = {}".format(epoch+1))
    print("Validation Accuracy = {}".format(vScore))
    print("Accuracy = {}".format(running_accuracy/len(trainLoader)))
    print("Loss = {}".format(running_loss))
    epochList.append(epoch+1)
    accList.append(running_accuracy/len(trainLoader))
    valList.append(vScore)
    lossList.append(running_loss)


print('Finished Training')

Code for drawing graphs

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochList,lossList)
plt.title("Epoch vs Loss MLP")
plt.show()

plt.plot(accList,label="Train Accuracy")
plt.plot(valList,label="Validation Accuracy")
plt.title("Epoch vs Accuracy MLP")
plt.xlabel('No. epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Code for base CNN model

In [ ]:
def getValScoreNN(valLoader):
  correct = 0
  total = 0
  # Since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in valLoader:
          i = i+1
          images, labels = data
          # calculate outputs by running images through the network
          outputs = net(images)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          #Adds to total counter
          total += labels.size(0)
          #If correct add to counter
          correct += (predicted == labels).sum().item()
  return correct/total

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #3 channels for RGB to 6 channels with kernel size 5x5
        self.pool = nn.MaxPool2d(2, 2) #Max pooling with size of 2x2
        self.conv2 = nn.Conv2d(6, 16, 5) #6 channels from above to 16 channels with kernel size 5x5
        self.fc1 = nn.Linear(1296, 120) #Linear layer to reduce layers to 120
        self.fc2 = nn.Linear(120, 84) #Reduce layers to 84
        self.fc3 = nn.Linear(84, 3) #Reduce to the number of classes being 3

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #Applies the first convolution layer with ReLu and apply Max pooling to that
        #print(x.size())
        x = self.pool(F.relu(self.conv2(x))) #Applies the second convolution layer with ReLu and apply Max pooling to that
        #print(x.size())
        x = torch.flatten(x, 1) # Flattens all dimensions except batch
        #print(x.size())
        x = F.relu(self.fc1(x)) #Applies Linear layer with ReLu
        x = F.relu(self.fc2(x)) #Applies Linear layer with ReLu
        x = self.fc3(x) #Gets the class
        return x


net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9)

In [ ]:
epochListNN = []
valListNN = []
accListNN = []
lossListNN = []
for epoch in range(20):  # loop over the dataset multiple times
    running_accuracy=0.0
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients  as they typically accumulate
        optimizer.zero_grad()

        # forward + backward + optimize
        #Get outputs from current model
        outputs = net(inputs)
        #Calculate Loss
        loss = criterion(outputs, labels)
        #Runs autograd to fill in gradients of optimizer using loss
        loss.backward()
        #Updates optimizer parameters
        optimizer.step()

        # print statistics
        
        _, predicted = torch.max(outputs.data, 1)
        running_accuracy += accuracy(outputs, labels)
    running_loss += loss.item()
    vScore = getValScoreNN(valLoader)
    print("Epoch = {}".format(epoch+1))
    print("Validation Accuracy = {}".format(vScore))
    print("Accuracy = {}".format(running_accuracy/len(trainLoader)))
    print("Loss = {}".format(running_loss))
    epochListNN.append(epoch+1)
    accListNN.append(running_accuracy/len(trainLoader))
    valListNN.append(vScore)
    lossListNN.append(running_loss)

print('Finished Training')

In [ ]:
plt.plot(epochListNN,lossListNN)
plt.title("Epoch vs Loss CNN")
plt.show()

plt.plot(accListNN,label="Train Accuracy")
plt.plot(valListNN,label="Validation Accuracy")
plt.title("Epoch vs Accuracy CNN")
plt.xlabel('No. epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Final Model Using VGG

In [32]:
def getValScoreVGG(valLoader):
  correct = 0
  total = 0
  # Since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in valLoader: #Iterates through validation set
          images, labels = data
          # Calculate outputs by running images through the network
          outputs = model(images)
          # The class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          #Counts images seen
          total += labels.size(0)
          #Adds to correct counter if the predicted label is correct
          correct += (predicted == labels).sum().item()
  return correct/total

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
 
#Uses torches torch.hub repository to get the vgg19 model and set pretrained to true to use ImageNet pretrained weights
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True)
#Use Cross Entropy so no need to softmax
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

#Resets lists for storing values at each epoch
epochListVGG = []
valListVGG = []
accListVGG = []
lossListVGG = []
for epoch in range(7):  # loop over the dataset 7 times
    running_accuracy=0.0
    running_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        # get the inputs; data is a list of [inputs, labels] labels is 0,1 or 2 for class and input is tensor image
        inputs, labels = data

        # zero the parameter gradients as they typically accumulate
        optimizer.zero_grad()

        # forward + backward + optimize
        #Get outputs from current model
        outputs = model(inputs)
        #Calculate Loss
        loss = criterion(outputs, labels)
        #Runs autograd to fill in gradients of optimizer using loss
        loss.backward()
        #Updates optimizer parameters
        optimizer.step()
        
        #calculates accuracy for each batch
        running_accuracy += accuracy(outputs, labels) 

    #Calculates and prints statistics
    running_loss += loss.item()
    vScore = getValScoreVGG(valLoader)
    print("Epoch = {}".format(epoch+1))
    print("Validation Accuracy = {}".format(vScore))
    print("Accuracy = {}".format(running_accuracy/len(trainLoader)))
    print("Loss = {}".format(running_loss))
    epochListVGG.append(epoch+1)
    accListVGG.append(running_accuracy/len(trainLoader))
    valListVGG.append(vScore)
    lossListVGG.append(running_loss)

print('Finished Training')


Saving the VGG model

In [6]:
 torch.save(model.state_dict(), './model.pth')

VGG19 consists of Convolution 3x3 layers (to detect features in input) with stride 1,1 and padding 1,1,1,1 (to keep spatial resolution) then ReLu layer (non-linearity to reduce training time and classify better by smoothing and making more distinct boundaries) followed by Max Pooling (reduce height and width and fix distorted images) and then a few Fully Connected layers with ReLu (process data into NN through linear rectifier activation function it increases non-linearity) and 50% dropout (to reduce overfitting by dropping units randomly and connections to reduce dependency on one unit) and finally a SoftMax layer (expontentiates logits to make posistive and scale so sum to 1 to make them probabilities not really needed since using Cross Enthropy). 

The model is pretrained using ImageNet which is a collection of over 14million images which makes for a smart model to start with. 

Graphs for VGG model

In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochListVGG,lossListVGG)
plt.title("Epoch vs Loss VGG")
plt.show()  

plt.plot(epochListVGG,accListNN,label="Train Accuracy")
plt.plot(epochListVGG,valListNN,label="Validation Accuracy")
plt.title("Epoch vs Accuracy VGG")
plt.xlabel('No. epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Loading Saved Final Model and testing validation accuracy

In [59]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True)
model.load_state_dict(torch.load('./VGG-Fold1-250.pth'))

correct = 0
total = 0
  # Since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in valLoader:
        images, labels = data
          # calculate outputs by running images through the network
        outputs = model(images)
          # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        #Adds to total
        total += labels.size(0)
        #Adds to correct total if predicted label is correct
        correct += (predicted == labels).sum().item()
print(correct/total)

Using cache found in C:\Users\jepey/.cache\torch\hub\pytorch_vision_v0.10.0


0.9721595184349134
